<a href="https://colab.research.google.com/github/Earlyyyyy/Automatic-License-Plate-Detection/blob/main/model_car_plate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'car-plate-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F686454%2F1203932%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241011%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241011T172755Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D535d0b52fac8f4d6fc1f4117176420285ac030c089c3f809bf8228bfb0124ee6be605880fed7fbd354078039a00a036169fb41d7a596add97665cc576ab3d188edb6843d0b0ea3e14f6c2aac6149a8e6215f3d7bd083c3a21f2451975819af4ed21a568b5612e3ca95dbd054d6e0e0538eaa4ddbd8c27fe25da78b92ba852e098d6e05e8abd35ee40adb09f444e97e0fc78131cf2bd57f56c9072438764e3773c9590e11a86e0c6e08a52757aa3bc6eb84dd373f079b388047d1822ceae2f1f2974f6500e893c625e5341248810d81201c062f1266adbb79dc06771c4229a318b9286a2f64bcde6fefee5fc22a78ba52edaebda3d9f05b7635da6ed7611a0fbf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 212910636 bytes downloaded
Downloaded and uncompressed: car-plate-detection
Data source import complete.


In [31]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet

from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

## Parsing data from XML file

In [8]:
# /kaggle/input/car-plate-detection/input/car-plate-detection
path = glob('/kaggle/input/car-plate-detection/annotations/*.xml')
labels_dict = dict(filepath=[], img = [],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_img_name = root.find('filename').text
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)
    labels_dict['img'].append(member_img_name)

In [9]:
df = pd.DataFrame(labels_dict)
df = df.sort_values(by=['filepath'])
df = df.reset_index(drop=True)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,img,xmin,xmax,ymin,ymax
0,/kaggle/input/car-plate-detection/annotations/...,Cars0.png,226,419,125,173
1,/kaggle/input/car-plate-detection/annotations/...,Cars1.png,134,262,128,160
2,/kaggle/input/car-plate-detection/annotations/...,Cars10.png,140,303,5,148
3,/kaggle/input/car-plate-detection/annotations/...,Cars100.png,175,214,114,131
4,/kaggle/input/car-plate-detection/annotations/...,Cars101.png,167,240,202,220


In [10]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/kaggle/input/car-plate-detection/images',filename_image)
    return filepath_image

In [11]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]#

['/kaggle/input/car-plate-detection/images/Cars0.png',
 '/kaggle/input/car-plate-detection/images/Cars1.png',
 '/kaggle/input/car-plate-detection/images/Cars10.png',
 '/kaggle/input/car-plate-detection/images/Cars100.png',
 '/kaggle/input/car-plate-detection/images/Cars101.png',
 '/kaggle/input/car-plate-detection/images/Cars102.png',
 '/kaggle/input/car-plate-detection/images/Cars103.png',
 '/kaggle/input/car-plate-detection/images/Cars104.png',
 '/kaggle/input/car-plate-detection/images/Cars105.png',
 '/kaggle/input/car-plate-detection/images/Cars106.png']

## Verify data

In [12]:
image_path[87]

'/kaggle/input/car-plate-detection/images/Cars177.png'

In [13]:
df[df['img'] == 'Cars177.png']

,filepath,img,xmin,xmax,ymin,ymax
87,/kaggle/input/car-plate-detection/annotations/...,Cars177.png,149,272,166,193


In [14]:
file_path = image_path[87] #path of our image N2.jpeg
img = cv2.imread(file_path) #read the image
# xmin-1804/ymin-1734/xmax-2493/ymax-1882
img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10),xaxis_title='Figure 8 - N2.jpeg with bounding box')
fig.add_shape(type='rect',x0=149, x1=272, y0=166, y1=193, xref='x', yref='y',line_color='cyan')

# 3. DATA PROCESSING

## 3.1 Read data

In [15]:
#Targeting all our values in array selecting all columns
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    img,xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

## 3.2 SPLIT TRAIN AND TEST SET

In [16]:
# Convert data to array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [17]:
# Split the data into training and testing set using sklearn.
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

# 4. DEEP LEARNING FOR OBJECT DETECTION

## 4.1 INCEPTION-RESNET-V2 MODEL BUILDING¶

In [18]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

219055592/219055592 [==============================] - 1s 0us/step


In [19]:
# Complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

## 4.2 INCEPTION-RESNET-V2 TRAINING AND SAVE

In [20]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=180,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/180
35/35 [==============================] - 93s 1s/step - loss: 0.0433 - val_loss: 0.0371
Epoch 2/180
35/35 [==============================] - 43s 1s/step - loss: 0.0125 - val_loss: 0.0228
Epoch 3/180
35/35 [==============================] - 43s 1s/step - loss: 0.0064 - val_loss: 0.0190
Epoch 4/180
35/35 [==============================] - 42s 1s/step - loss: 0.0061 - val_loss: 0.0295
Epoch 5/180
35/35 [==============================] - 42s 1s/step - loss: 0.0047 - val_loss: 0.0204
Epoch 6/180
35/35 [==============================] - 42s 1s/step - loss: 0.0030 - val_loss: 0.0204
Epoch 7/180
35/35 [==============================] - 42s 1s/step - loss: 0.0023 - val_loss: 0.0188
Epoch 8/180
35/35 [==============================] - 42s 1s/step - loss: 0.0020 - val_loss: 0.0190
Epoch 9/180
35/35 [==============================] - 42s 1s/step - loss: 0.0019 - val_loss: 0.0203
Epoch 10/180
35/35 [==============================] - 42s 1s/step - loss: 0.0018 - val_loss: 0.0160
Epoch 11/

In [21]:
model.save('./object_detection.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.

